### Imports and Flag

In [3]:
import os
import pandas as pd
import polars as pl
import numpy as np

### Environment Setup ###
IS_KAGGLE = False  # Flag to switch between environments

### Submit

In [4]:
# Setup paths based on environment
if IS_KAGGLE:
    data_path = '/kaggle/input/jane-street-real-time-market-data-forecasting'
else:
    # For local testing
    import sys
    # Path to main project directory
    PROJ_DIR = os.path.dirname(os.getcwd())
    
    # Add paths to system path
    sys.path.append(os.path.join(PROJ_DIR, "jane-street-real-time-market-data-forecasting"))
    sys.path.append(os.path.join(PROJ_DIR, "training", "src", "utils"))
    
    # Set data path to our local test data
    data_path = os.path.join(os.getcwd(), "local_test_data")
    
    # Import local testing metrics
    from metrics import r2_score_weighted

import kaggle_evaluation.jane_street_inference_server

# Global variable to store lags
lags_ : pl.DataFrame | None = None

def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    global lags_
    
    if lags is not None:
        lags_ = lags

    # Simple prediction
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )    
    
    assert isinstance(predictions, pl.DataFrame | pd.DataFrame) # The predict function must return a DataFrame
    assert predictions.columns == ['row_id', 'responder_6']     # with columns 'row_id', 'responer_6'
    assert len(predictions) == len(test)                        # and as many rows as the test data.

    return predictions

# Set up inference server
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

# Run based on environment
if os.getenv('KAGGLE_IS_COMPETITION_RERUN') or IS_KAGGLE:
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            os.path.join(data_path, "test.parquet"),
            os.path.join(data_path, "lags.parquet"),
        )
    )
    
    # Local scoring
    print("Scoring submission...")
    predictions = pl.read_parquet("submission.parquet")                      # Read submission predictions
    test_data = pl.read_parquet(os.path.join(data_path, "test.parquet"))     # Read test data with actual values

    # Score only rows marked for scoring
    mask = test_data['is_scored']
    score = r2_score_weighted(
        test_data.filter(mask)['responder_6'].to_numpy(),
        predictions.filter(mask)['responder_6'].to_numpy(),
        test_data.filter(mask)['weight'].to_numpy()
    )
    print(f"Overall R² score: {score}")

Scoring submission...
Overall R² score: -1.4066840936521885e-07
